In [19]:
# CLAUSULAS SUELO
correo1 = [
    """Estimados señores:

    Me dirijo a ustedes como cliente de su entidad desde hace más de diez años. Tras revisar la información sobre mi préstamo hipotecario,
    he detectado que en el contrato figura una cláusula suelo que ha limitado la bajada de intereses, a pesar de la evolución del Euríbor en los últimos años.

    Solicito, por tanto, que se me informe detalladamente sobre la existencia de dicha cláusula, así como las medidas que su entidad tiene previstas para proceder a la devolución de las cantidades cobradas de más.

    Quedo a la espera de su pronta respuesta.
    """]

# COSTES HIPOTECA
correo2 = [

    """Buenos días:

    Me pongo en contacto con ustedes para presentar una reclamación relativa a los gastos de constitución de mi hipoteca formalizada en el año 2015.
    Según recientes sentencias judiciales, entiendo que determinados costes como la notaría, el registro de la propiedad y la gestoría no debieron ser asumidos íntegramente por mí.

    Solicito que se revise mi expediente y se me reintegren los importes correspondientes, junto con los intereses que legalmente procedan.
    """]

# FRAUDE BANCARIO
correo3 =[

    """Estimados,

    Hoy he detectado varios cargos en mi cuenta corriente que no reconozco ni he autorizado. Se trata de tres movimientos realizados entre los días 14 y 16 de junio,
    con importes superiores a los 300 euros cada uno. Ya he procedido a bloquear mi tarjeta, pero me preocupa que se haya producido un acceso fraudulento a mi cuenta.

    Les ruego que inicien una investigación de inmediato y me informen sobre los pasos a seguir para la recuperación de los importes sustraídos.
    """
]

# Etiquetas posibles (temáticas)
categorias = [
    "Cláusula suelo",
    "Costes hipotecarios",
    "Fraude bancario",
    "Tarjetas de crédito o débito",
    "Comisiones bancarias",
    "Problemas con transferencias",
    "Solicitud de cancelación de cuenta",
    "Errores en recibos domiciliados",
    "Actualización de datos personales",
    "Otros"
]

# PRUEBA 1

In [18]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification

MODEL_NAME = "roberta-large-mnli"
device     = "cuda" if torch.cuda.is_available() else "cpu"
template   = "Este correo trata sobre {}."


tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model     = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME).to(device)
model.eval()

def prob_entailment(premise: str, hypothesis: str) -> float:
    """Devuelve la probabilidad (entre 0-1) de que el modelo NLI
       considere que la premisa implica la hipótesis."""
    inputs = tokenizer.encode_plus(premise,
                                   hypothesis,
                                   return_tensors="pt",
                                   truncation=True,
                                   max_length=512).to(device)
    with torch.no_grad():
        logits = model(**inputs).logits.squeeze()          # [contradiction, neutral, entailment]
    probs  = torch.softmax(logits, dim=-1).cpu().numpy()
    return float(probs[2])    #


correos_dict = {"Correo 1": correo1, "Correo 2": correo2, "Correo 3": correo3}
matriz = {}

for nombre, lista in correos_dict.items():
    texto = "\n\n".join(lista)
    fila  = {}
    for cat in categorias:
        hypothesis = template.format(cat)
        p_entail   = prob_entailment(texto, hypothesis)
        fila[cat]  = round(p_entail, 4)
    matriz[nombre] = fila

df = pd.DataFrame(matriz).T      # filas = correos, columnas = categorías
print(df)

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


          Cláusula suelo  Costes hipotecarios  Fraude bancario  \
Correo 1          0.8417               0.5285           0.1009   
Correo 2          0.0748               0.6377           0.0503   
Correo 3          0.0549               0.1999           0.0367   

          Tarjetas de crédito o débito  Comisiones bancarias  \
Correo 1                        0.2139                0.0994   
Correo 2                        0.2679                0.0705   
Correo 3                        0.1044                0.0605   

          Problemas con transferencias  Solicitud de cancelación de cuenta  \
Correo 1                        0.2318                              0.3543   
Correo 2                        0.5364                              0.3913   
Correo 3                        0.2882                              0.2329   

          Errores en recibos domiciliados  Actualización de datos personales  \
Correo 1                           0.2801                             0.4217   
Corre

In [20]:
categoria_predicha = df.idxmax(axis=1)
score_max          = df.max(axis=1)
categoria_real     = ["Cláusula suelo", "Costes hipotecarios", "Fraude bancario"]

df_pred = pd.DataFrame({
    "Categoria_predicha": categoria_predicha,
    "Score_max": score_max.round(4),
    "Categoria_real": categoria_real
})

print(df_pred)

                         Categoria_predicha  Score_max       Categoria_real
Correo 1                     Cláusula suelo     0.8417       Cláusula suelo
Correo 2                Costes hipotecarios     0.6377  Costes hipotecarios
Correo 3  Actualización de datos personales     0.5269      Fraude bancario


# PRUEBA 2

In [21]:
def prob_entailment_sep(premise: str, hypothesis: str) -> float:
    sep_token = tokenizer.sep_token
    joint_text = f"{premise} {sep_token} {hypothesis}"

    inputs = tokenizer(
        joint_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    ).to(device)

    with torch.no_grad():
        logits = model(**inputs).logits.squeeze()
    probs = torch.softmax(logits, dim=-1).cpu().numpy()  # [contradiction, neutral, entailment]
    return float(probs[2])

In [22]:
df_probs = {}

for nombre, lista in correos_dict.items():
    premise = "\n\n".join(lista)
    fila = {}
    for cat in categorias:
        hyp = template.format(cat)
        fila[cat] = round(prob_entailment_sep(premise, hyp), 4)
    df_probs[nombre] = fila

df_probs = pd.DataFrame(df_probs).T
print("Matriz de probabilidades:")
print(df_probs)

Matriz de probabilidades:
          Cláusula suelo  Costes hipotecarios  Fraude bancario  \
Correo 1          0.8044               0.5139           0.1076   
Correo 2          0.0838               0.5944           0.0709   
Correo 3          0.0492               0.1760           0.0414   

          Tarjetas de crédito o débito  Comisiones bancarias  \
Correo 1                        0.2245                0.1040   
Correo 2                        0.2337                0.0936   
Correo 3                        0.1157                0.0638   

          Problemas con transferencias  Solicitud de cancelación de cuenta  \
Correo 1                        0.2404                              0.3497   
Correo 2                        0.4538                              0.3798   
Correo 3                        0.2332                              0.2058   

          Errores en recibos domiciliados  Actualización de datos personales  \
Correo 1                           0.2851                  

In [23]:
df_final = pd.DataFrame({
    "Categoria_predicha": df_probs.idxmax(axis=1),
    "Score_max":          df_probs.max(axis=1).round(4),
    "Categoria_real": categoria_real
})
print("Output definitivo:")
print(df_final)

Output definitivo:
                         Categoria_predicha  Score_max       Categoria_real
Correo 1                     Cláusula suelo     0.8044       Cláusula suelo
Correo 2                Costes hipotecarios     0.5944  Costes hipotecarios
Correo 3  Actualización de datos personales     0.4583      Fraude bancario
